# Imports

In [24]:
import nfl_data_py as nfl
import pandas as pd

# Grab Data

In [25]:
years = [2023]

In [31]:
df_weekly = nfl.import_weekly_data(years)

Downcasting floats.


In [33]:
df_pbp = nfl.import_pbp_data(years)

2023 done.
Downcasting floats.


# Selecting Position For SoS

In [44]:
position = 'QB'

In [45]:
df_position = df_weekly.query(f"position == '{position}' and season_type == 'REG'")

# Calculating Defensive Strength

In [105]:
def_columns = ["Team", "Fantasy Points Allowed", "Score", "Week", "Home/Away", "QB", "Offense",
               'Completions', 'Attempts', 'Passing Yards Allowed', 'Passing Touchdowns Allowed',
               'Interceptions', 'Sacks', "Fumbles Recovered", "Fumbles Forced", 'Passing Air Yards',
               'PACR', 'Dakota', "Roof"
              ]
df_defense_rank_weekly = pd.DataFrame(columns=def_columns)

In [106]:
# higher pacr is better for QB -> means more passing yards achieved of thrown passing air yards

In [107]:
# higher Dakota is better for QB -> More EPA or CPOE

In [108]:
for i, row in df_position.iterrows():
    player_name = row["player_name"]
    player_id = row["player_id"]
    
    off_team = row["recent_team"]
    def_team = row["opponent_team"]
    fantasy_points = row["fantasy_points"]
    week = row["week"]
    
    # for the composite score - assigning variables, defaulting to 0 if NaN
    completions = 0 if pd.isna(row["completions"]) else row["completions"]
    attempts = 0 if pd.isna(row["attempts"]) else row["attempts"]
    passing_yards = 0 if pd.isna(row["passing_yards"]) else row["passing_yards"]
    passing_tds = 0 if pd.isna(row["passing_tds"]) else row["passing_tds"]
    ints = 0 if pd.isna(row["interceptions"]) else row["interceptions"]
    sacks = 0 if pd.isna(row["sacks"]) else row["sacks"]
    fumbles_recovered = (0 if pd.isna(row["sack_fumbles_lost"]) else row["sack_fumbles_lost"]) + (0 if pd.isna(row["rushing_fumbles_lost"]) else row["rushing_fumbles_lost"])
    fumbles_forced = (0 if pd.isna(row["sack_fumbles"]) else row["sack_fumbles"]) + (0 if pd.isna(row["rushing_fumbles"]) else row["rushing_fumbles"])
    passing_a_y = 0 if pd.isna(row["passing_air_yards"]) else row["passing_air_yards"]
    pacr = 0 if pd.isna(row["pacr"]) else row["pacr"]
    dakota = 0 if pd.isna(row["dakota"]) else row["dakota"]
    rushing_yards = 0 if pd.isna(row["rushing_yards"]) else row["rushing_yards"]
    rushing_tds = 0 if pd.isna(row["rushing_tds"]) else row["rushing_tds"]
    
    # composite score calc:
    composite_score = (completions * -0.02) + (attempts * -0.01) + (passing_yards * -0.04) + (passing_tds * -5) \
        + (ints * 2) + (sacks * 1) + (fumbles_recovered * 2) + (fumbles_forced * 0.3) + (passing_a_y * -0.01) \
        + (pacr * -10) + (dakota * -5) 
#         + (rushing_tds * -6) + (rushing_yards * -0.10)
    
    # final score (subtract fantasy points - they were allowed by the defense)
#     score = composite_score - fantasy_points
    score = composite_score
    
    # other misc details
    home_away_def = ""
    roof = ""
    df_plays_for_week = df_pbp.query(f"passer_player_id == '{player_id}' and week == {week}")
    if not df_plays_for_week.empty:
        home_team = df_plays_for_week["home_team"].iloc[0]
        away_team = df_plays_for_week["away_team"].iloc[0]
        roof = df_plays_for_week["roof"].iloc[0]

        if def_team == home_team:
            home_away_def = "Home"
        else:
            home_away_def = "Away"
    
    new_row = {
        "Team": def_team,
        "Fantasy Points Allowed": fantasy_points,
        "Score": score,
        "Week": week,
        "Home/Away": home_away_def,
        "QB": player_name,
        "Offense": off_team,
        'Completions': completions,
        'Attempts': attempts,
        'Passing Yards Allowed': passing_a_y,
        'Passing Touchdowns Allowed': passing_tds,
        'Interceptions': ints,
        'Sacks': sacks,
        "Fumbles Recovered": fumbles_recovered,
        "Fumbles Forced": fumbles_forced,
        'Passing Air Yards': passing_a_y,
        'PACR': pacr,
        'Dakota': dakota,
        'Roof': roof
    }
    
    new_row_df = pd.DataFrame([new_row])
    df_defense_rank_weekly = pd.concat([df_defense_rank_weekly, new_row_df], ignore_index=True)

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_24303/2661945140.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_defense_rank_weekly = pd.concat([df_defense_rank_weekly, new_row_df], ignore_index=True)


In [109]:
df_defense_rank_weekly.sort_values(by="Week")

,Team,Fantasy Points Allowed,Score,Week,Home/Away,QB,Offense,Completions,Attempts,Passing Yards Allowed,Passing Touchdowns Allowed,Interceptions,Sacks,Fumbles Recovered,Fumbles Forced,Passing Air Yards,PACR,Dakota,Roof
0,BUF,0.000000,0.820000,1,Away,A.Rodgers,NYJ,0,1,17.0,0,0.0,1.0,0.0,0.0,17.0,0.000000,0,outdoors
537,PIT,16.799999,-26.309561,1,Home,B.Purdy,SF,19,29,242.0,2,0.0,3.0,1.0,2.0,242.0,0.909091,0.18573,outdoors
520,ARI,15.180000,-14.548035,1,Away,S.Howell,WAS,19,31,228.0,1,1.0,6.0,1.0,1.0,228.0,0.885965,-0.012323,outdoors
508,BUF,8.200000,-22.978011,1,Away,Z.Wilson,NYJ,14,21,96.0,1,1.0,2.0,0.0,0.0,96.0,1.458333,0.068935,outdoors
54,TB,16.459999,-31.108301,1,Away,K.Cousins,MIN,33,44,298.0,2,1.0,2.0,2.0,2.0,298.0,1.154362,0.064935,dome
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,TB,4.160000,-8.763839,18,Away,B.Young,CAR,11,18,105.0,0,0.0,3.0,1.0,1.0,105.0,0.895238,-0.019708,outdoors
618,DEN,17.860001,-32.578936,18,Away,A.O'Connell,LV,20,31,227.0,2,0.0,2.0,0.0,0.0,227.0,1.074890,0.218007,dome
496,TEN,16.200001,-30.717968,18,Home,T.Lawrence,JAX,29,43,308.0,2,2.0,0.0,0.0,0.0,308.0,0.909091,0.067412,outdoors
319,MIA,23.059999,-30.462155,18,Home,J.Allen,BUF,30,38,368.0,2,2.0,3.0,1.0,1.0,368.0,0.975543,0.197344,outdoors


In [110]:
df_seasonal_scores = df_defense_rank_weekly.copy().groupby(["Team"]).agg({"Score": "mean"})

In [119]:
df_def_avg = df_seasonal_scores.sort_values(by="Score", ascending=False)

In [120]:
df_def_avg

,Score
Team,
NYJ,-15.757591
BAL,-16.751721
CLE,-17.005123
DAL,-17.335853
CAR,-18.065653
NE,-18.287097
IND,-18.643557
ATL,-18.776083
LA,-20.318886


In [113]:
df_qb_scores = df_defense_rank_weekly.copy().groupby(["QB"]).agg({"Score": "mean", "Fantasy Points Allowed": "mean"})

In [114]:
df_sos_by_qb = df_qb_scores.sort_values(by="Fantasy Points Allowed")[-30:]

In [115]:
df_sos_by_qb.sort_values(by="Score")

,Score,Fantasy Points Allowed
QB,,
J.Flacco,-35.343644,20.168000
K.Cousins,-33.315687,18.717500
B.Purdy,-33.163381,18.475000
D.Prescott,-32.328900,20.167059
T.Tagovailoa,-31.829825,16.021177
J.Goff,-31.348875,17.005882
P.Mahomes,-30.627392,17.513750
M.Stafford,-29.482471,16.206667
J.Love,-28.908221,18.768235


In [116]:
df_sos_by_qb["Score"] = df_sos_by_qb["Score"] + 40

In [124]:
df_sos_by_qb

,Score,Fantasy Points Allowed
QB,,
E.Stick,19.718846,13.512000
J.Driskel,20.515354,13.940000
D.Carr,13.134676,14.183529
D.Watson,21.729782,14.466666
J.Burrow,13.406826,14.716000
G.Smith,13.269633,15.084000
S.Howell,18.803269,15.090588
J.Dobbs,21.151362,15.589231
J.Browning,13.530975,15.937778


In [118]:
df_sos_by_qb.to_pickle("./df_sos_by_qb_passing_23")

In [121]:
df_def_avg["Score"] = df_def_avg["Score"] + 30

In [125]:
df_def_avg.to_pickle("./df_def_rating_by_qb_passing_23")

In [134]:
s = "1"
s[1:]

''

In [135]:
s.index("1")

0

In [138]:
s[0].

str